# Relating protein structure and post-translational modifications

The goal is to characterize whether various types of post-translational modifications (PTMs) occur more frequently in intrinsically disordered regions (IDRs) of proteins (1). The target of inference is the odds ratio associating being in an IDR and having a PTM.
This notebook shows how to do inference on the odds ratio via PPI by using structures predicted by AlphaFold (2) to predict IDRs. IDRs are predicted from structures following the strategy of Bludau et al. (3).

1. L. M. Iakoucheva, P. Radivojac, C. J. Brown, T. R. O’Connor, J. G. Sikes, Z. Obradovic, A. K. Dunker, The importance of intrinsic disorder for protein phosphorylation. Nucleic Acids Res. 32, 1037–1049 (2004).
2. J. Jumper, R. Evans, A. Pritzel, T. Green, M. Figurnov, O. Ronneberger, K. Tunyasuvunakool, R. Bates, A. Žídek, A. Potapenko, A. Bridgland, C. Meyer, S. A. A. Kohl, A. J. Ballard, A. Cowie, B. Romera-Paredes, S. Nikolov, R. Jain, J. Adler, T. Back, S. Petersen, D. Reiman, E. Clancy, M. Zielinski, M. Steinegger, M. Pacholska, T. Berghammer, S. Bodenstein, D. Silver, O. Vinyals, A. W. Senior, K. Kavukcuoglu, P. Kohli, D. Hassabis, Highly accurate protein structure prediction with AlphaFold. Nature 596(7873), 583–589 (2021). 
3. I. Bludau, S. Willems, W-F. Zeng, M. T. Strauss, F. M. Hansen, M. C. Tanzer, O. Karayel, B. A. Schulman, M. Mann, The structural context of posttranslational modifications at a proteome-wide scale. PLoS Biology 20(5), e3001636 (2022).

### Import necessary packages

In [1]:
import numpy as np
from datasets import load_dataset
from FL_cpp_method import analyze_dataset, plot_cpp
import pandas as pd
import os

### Import the AlphaFold data set

Load the data. The data set contains true indicators of disorder (```Y```), predicted indicators of disorder (```Yhat```), and indicators of a PTM (```phosphorylated```, ```ubiquitinated```, or ```acetylated```). Predictions of disorder are made based on AlphaFold predictions of structure.

In [2]:
dataset_name = "alphafold"
data = load_dataset('../data/', dataset_name)
Y_total = data["Y"]
Yhat_total = data["Yhat"]
Z = data["phosphorylated"].astype(
    bool
)  # Can choose from "phosphorylated", "ubiquitinated", or "acetylated"
# method_name = "ppi_mean_ci"

dataset_dist = 'IID'
# dataset_dist = 'Non-IID'

alpha = 0.1

method = 'mean'

Y0_total, Y1_total = Y_total[~Z], Y_total[Z]
Yhat0_total, Yhat1_total = Yhat_total[~Z], Yhat_total[Z]

### Problem setup

Specify the error level (```alpha```), range of values for the labeled data set size (```ns```), and number of trials (```num_trials```).

Compute the ground-truth value of the estimand.

In [3]:
# num_ratio = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
num_ratio = [1, 1, 1, 1, 1]  # 数据量分布平衡
# num_ratio = [1,2,3,3,1]  # 数据量分布不平衡

# 计算标注真实值、各节点上、组合数据后和FL后的平均值cpp
true_theta0, cpp_intervals0, ppi_ci_combined0, mean_cpp0 = analyze_dataset(alpha, None, Y0_total, Yhat0_total, dataset_dist,
                                                                            num_ratio, method, grid=None)

true_theta1, cpp_intervals1, ppi_ci_combined1, mean_cpp1 = analyze_dataset(alpha, None, Y1_total, Yhat1_total, dataset_dist,
                                                                            num_ratio, method, grid=None)

# True odds ratio
true_theta = (true_theta1 / (1 - true_theta1)) / (true_theta0 / (1 - true_theta0))

分组： 1
带标签的样本量： 95
不带标签的样本量： 862
分组： 2
带标签的样本量： 95
不带标签的样本量： 862
分组： 3
带标签的样本量： 95
不带标签的样本量： 862
分组： 4
带标签的样本量： 95
不带标签的样本量： 862
分组： 5
带标签的样本量： 95
不带标签的样本量： 862
imputed var: [1.1335261e-05]
rectifier var [7.69587847e-05]
带标签的样本量： 475
不带标签的样本量： 4310

最终结果：
真实 theta: 0.11912225705329153
CPP intervals: [array([0.06252746, 0.12001637]), array([0.08151825, 0.16905854]), array([0.07121033, 0.14974207]), array([0.05489065, 0.11701496]), array([0.07334583, 0.1274396 ])]
组合数据的置信区间: [0.08684403 0.11796909]
联邦聚合后的置信区间: [0.08722055 0.11813226]
分组： 1
带标签的样本量： 120
不带标签的样本量： 1084
分组： 2
带标签的样本量： 120
不带标签的样本量： 1084
分组： 3
带标签的样本量： 120
不带标签的样本量： 1083
分组： 4
带标签的样本量： 120
不带标签的样本量： 1083
分组： 5
带标签的样本量： 120
不带标签的样本量： 1083
imputed var: [1.72441314e-05]
rectifier var [8.56684913e-05]
带标签的样本量： 600
不带标签的样本量： 5417

最终结果：
真实 theta: 0.22369951803224197
CPP intervals: [array([0.16109428, 0.22712209]), array([0.16068777, 0.2320239 ]), array([0.19818896, 0.28671347]), array([0.19354044, 0.26900797]), array([0.17905923, 

### Construct intervals

Form confidence intervals for all methods and problem parameters. A dataframe with the following columns is formed:
1. ```method``` (one of ```PPI```, ```Classical```, and ```Imputation```)
2. ```n``` (labeled data set size, takes values in ```ns```)
3. ```lower``` (lower endpoint of the confidence interval)
4. ```upper``` (upper endpoint of the confidence interval)
5. ```trial``` (index of trial, goes from ```0``` to ```num_trials-1```)

In [4]:
def odds_ratio_ci(mu0_ci, mu1_ci):
    # First construct CI of mu0/(1-mu0) and mu1/(1-mu1)
    r0 = [mu0_ci[0] / (1 - mu0_ci[0]), mu0_ci[1] / (1 - mu0_ci[1])]
    r1 = [mu1_ci[0] / (1 - mu1_ci[0]), mu1_ci[1] / (1 - mu1_ci[1])]
    return r1[0] / r0[1], r1[1] / r0[0]

In [5]:
cpp_intervals = []
for i in range(len(cpp_intervals0)):
    tmp = odds_ratio_ci(cpp_intervals0[i], cpp_intervals1[i])
    print(tmp)
    cpp_intervals.append(np.array(tmp))

ppi_ci_combined = odds_ratio_ci(ppi_ci_combined0, ppi_ci_combined1)

mean_cpp = odds_ratio_ci(mean_cpp0, mean_cpp1)

(1.4079948278656798, 4.405916207828402)
(0.9410064054363249, 3.4040873881728992)
(1.4035060756920532, 5.242742051327938)
(1.81092765850092, 6.336307136954414)
(1.4933993543706907, 4.164994031594971)


### Plot results

Plot:
1. Five randomly chosen intervals from the dataframe for PPI and the classical method, and the imputed interval;
2. The average interval width for PPI and the classical method, together with a scatterplot of the widths from the five random draws.

In [6]:
print("true theta", true_theta)
print("FL",mean_cpp)
print("组合数据", ppi_ci_combined)
print('Client', cpp_intervals)
# # 文件名
# filename = 'parameters.csv'
# 
# # 新的数组数据
# new_data = {
#     'centralized_lower': [ppi_ci_combined[0]],
#     'centralized_upper': [ppi_ci_combined[1]],
#     'fl_lower': [mean_cpp[0]],
#     'fl_upper': [mean_cpp[1]]
# }
# 
# # 检查参数文件是否存在
# if os.path.exists(filename):
#     # 如果文件存在，读取现有数据
#     df = pd.read_csv(filename)
# else:
#     # 如果文件不存在，创建一个空的 DataFrame
#     df = pd.DataFrame(columns=['centralized_lower', 'centralized_upper', 'fl_lower', 'fl_upper'])
# 
# # 将新数据添加到 DataFrame
# new_df = pd.DataFrame(new_data)
# df = pd.concat([df, new_df], ignore_index=True)
# 
# # 保存更新后的数据到CSV文件
# df.to_csv(filename, index=False)
# 
# print(f"Updated CSV file '{filename}' successfully.")

file_name = dataset_dist + '-' + dataset_name + '.pdf'
xlim = [-4, 8]
ylim = [0, 1.0]
title = "odds ratio between \n disorder and phosphorylation"
plot_cpp(true_theta, cpp_intervals, ppi_ci_combined, mean_cpp, file_name, xlim, ylim, title)

true theta 2.1308747140812856
FL (1.8528049181948325, 3.1652007532718645)
组合数据 (1.8888405820322125, 3.228499162426632)
Client [array([1.40799483, 4.40591621]), array([0.94100641, 3.40408739]), array([1.40350608, 5.24274205]), array([1.81092766, 6.33630714]), array([1.49339935, 4.16499403])]
